<a href="https://colab.research.google.com/github/nichakornchaisuwan/Project_Boneage/blob/main/lightning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pytorch-lightning

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.3/800.3 KB 35.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.4/125.4 KB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.4/512.4 KB 45.9 MB/s eta 0:00:00


In [3]:
import pandas as pd
import numpy as np
from PIL import Image

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader, random_split
from torchvision import transforms

import pytorch_lightning as pl
#from pytorch_lightning.logging import TensorBoardLogger
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks.early_stopping import EarlyStopping

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
path = '/content/drive/My Drive/Project_Boneage'

In [6]:
#train = pd.read_csv(r"/content/drive/MyDrive/Project_Boneage/train.csv")

In [7]:
#train

In [8]:
#train['id'] = train['id'].apply(lambda x: str(x)+'.png')
#train['boneage'] = train['boneage'].apply(lambda x: str(x))
#train['male'] = train['male'].apply(lambda x: 'male' if x else 'female')

#df1 = pd.DataFrame(train)
#df1.rename(columns={'male':'gender'},inplace=True)

In [9]:
#train

#Creating a custom PyTorch Dataset

In [10]:
class ImageDataset(Dataset):
    """Tabular and Image dataset."""

    def __init__(self, pickle_file, image_dir):
        self.image_dir = image_dir
        self.pickle_file = pickle_file
        self.tabular = pd.read_pickle(pickle_file)

    def __len__(self):
        return len(self.tabular)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.tolist()

        tabular = self.tabular.iloc[idx, 0:]

        y = tabular["boneage"]

        image = Image.open(f"{self.image_dir}/{tabular['id']}.png")
        image = np.array(image)
        image = image[..., :3]

        image = transforms.functional.to_tensor(image)

        tabular = tabular[["gender"]]
        tabular = tabular.tolist()
        tabular = torch.FloatTensor(tabular)

        return image, tabular, y

In [11]:
def conv_block(input_size, output_size):
    block = nn.Sequential(
        nn.Conv2d(input_size, output_size, (3, 3)), nn.ReLU(), nn.BatchNorm2d(output_size), nn.MaxPool2d((2, 2)),
    )

    return block

In [12]:
class LitClassifier(pl.LightningModule):
    def __init__(
        self, lr: float = 1e-3, num_workers: int = 4, batch_size: int = 32,
    ):
        super().__init__()
        self.lr = lr
        self.num_workers = num_workers
        self.batch_size = batch_size

        self.conv1 = conv_block(3, 16)
        self.conv2 = conv_block(16, 32)
        self.conv3 = conv_block(32, 64)

        self.ln1 = nn.Linear(64 * 26 * 26, 16)
        self.relu = nn.ReLU()
        self.batchnorm = nn.BatchNorm1d(16)
        self.dropout = nn.Dropout2d(0.5)
        self.ln2 = nn.Linear(16, 5)

        self.ln4 = nn.Linear(5, 10)
        self.ln5 = nn.Linear(10, 10)
        self.ln6 = nn.Linear(10, 5)
        self.ln7 = nn.Linear(10, 1)

In [13]:
def forward(self, img, tab):
        img = self.conv1(img)
        img = self.conv2(img)
        img = self.conv3(img)
        img = img.reshape(img.shape[0], -1)
        img = self.ln1(img)
        img = self.relu(img)
        img = self.batchnorm(img)
        img = self.dropout(img)
        img = self.ln2(img)
        img = self.relu(img)

        tab = self.ln4(tab)
        tab = self.relu(tab)
        tab = self.ln5(tab)
        tab = self.relu(tab)
        tab = self.ln6(tab)
        tab = self.relu(tab)

        x = torch.cat((img, tab), dim=1)
        x = self.relu(x)

        return self.ln7(x)

#Training

In [14]:
def training_step(self, batch, batch_idx):
    image, tabular, y = batch

    criterion = torch.nn.L1Loss()
    y_pred = torch.flatten(self(image, tabular))
    y_pred = y_pred.double()

    loss = criterion(y_pred, y)

    tensorboard_logs = {"train_loss": loss}
    return {"loss": loss, "log": tensorboard_logs}

def validation_step(self, batch, batch_idx):
    image, tabular, y = batch

    criterion = torch.nn.L1Loss()
    y_pred = torch.flatten(self(image, tabular))
    y_pred = y_pred.double()

    val_loss = criterion(y_pred, y)

    return {"val_loss": val_loss}

def validation_epoch_end(self, outputs):
    avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
    tensorboard_logs = {"val_loss": avg_loss}
    return {"val_loss": avg_loss, "log": tensorboard_logs}

def test_step(self, batch, batch_idx):
    image, tabular, y = batch

    criterion = torch.nn.L1Loss()
    y_pred = torch.flatten(self(image, tabular))
    y_pred = y_pred.double()

    test_loss = criterion(y_pred, y)

    return {"test_loss": test_loss}

def test_epoch_end(self, outputs):
    avg_loss = torch.stack([x["test_loss"] for x in outputs]).mean()
    logs = {"test_loss": avg_loss}
    return {"test_loss": avg_loss, "log": logs, "progress_bar": logs}

In [15]:
    def setup(self, stage):

        image_datatrain = ImageDataset(pickle_file=f"{path}train.csv", image_dir=f"{path}boneage_training_dataset/")
        image_dataval = ImageDataset(pickle_file=f"{path}validation.csv", image_dir=f"{path}boneage_validation_dataset/")
        image_datatest = ImageDataset(pickle_file=f"{path}boneage-test-dataset.csv", image_dir=f"{path}boneage_test_dataset/")

        #train_size = int(0.80 * len(image_data))
        #val_size = int((len(image_data) - train_size) / 2)
        #test_size = int((len(image_data) - train_size) / 2)

        #self.train_set, self.val_set, self.test_set = random_split(image_data, (train_size, val_size, test_size))
        self.train_set = image_datatrain
        self.val_set = image_dataval
        self.test_set = image_datatest
        
    def configure_optimizers(self):
        return torch.optim.Adam(self.parameters(), lr=(self.lr))

    def train_dataloader(self):
        return DataLoader(self.train_set, batch_size=self.batch_size)

    def val_dataloader(self):
        return DataLoader(self.val_set, batch_size=self.batch_size)

    def test_dataloader(self):
        return DataLoader(self.test_set, batch_size=self.batch_size)

In [16]:
from fastai.vision.learner import Learner
from fastai.callback.schedule import lr_find

In [17]:
if __name__ == "__main__":
    logger = TensorBoardLogger("lightning_logs", name="image_only")
    early_stop_callback = EarlyStopping(monitor="val_loss",min_delta=5000,patience=7, verbose=False, mode="min")
    

In [19]:
model = LitClassifier()
trainer = pl.Trainer(accelerator='cpu', logger=logger, callbacks=early_stop_callback,auto_lr_find=True)


INFO:pytorch_lightning.utilities.rank_zero:GPU available: False, used: False
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [22]:
lr_finder = trainer.tuner.lr_find(model)
fig = lr_finder.plot(suggest=True, show=True)
new_lr = lr_finder.suggestion()
print(new_lr)
model.hparams.lr = new_lr

trainer.fit(model)
trainer.test(model)

ValueError: ignored

In [ ]:
trainer.fit(model)
trainer.test(model)

ValueError: ignored

In [ ]:
tensorboard --logdir=lightning_log

SyntaxError: ignored